In [1]:
import pymongo
from preprocessor.preprocessing import PreprocessPostContent
from multiprocessing import Pool


In [2]:
client = pymongo.MongoClient(host='10.1.1.9', port=50000)
javaposts_db = client.javaposts
posts_collection = javaposts_db.posts
posts = posts_collection.find()

In [3]:
print(posts.count())

390000


/home/LAB/caipb/mypyenv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


In [4]:
def process(post_bulk):
    para_list = []
    for post in post_bulk:
        para_list.extend(PreprocessPostContent().getPlainTxt(post['Title']))
        para_list.extend(PreprocessPostContent().getPlainTxt(post['Body']))

    return para_list

def parallel_process(posts):
    bulk = []
    block_list = []
    count = 0
    pool = Pool(50)
    for post in posts:
        bulk.append(post)
        count += 1
        if not count % 1000:
            block_list.append(bulk)
            bulk = []

    block_list.append(bulk)
    para_list = pool.map(process, block_list)
    pool.close()
    pool.join()
    return para_list

In [5]:
# used about 5 min
return_list = parallel_process(posts)
para_list = []

for ls in return_list:
    para_list.extend(ls)

In [8]:
len(para_list)
print(para_list[400])

I'm currently building a Java app that could end up being run on many different platforms , but primarily variants of Solaris , Linux and Windows .


In [29]:
# word2vec 接收的是单词列表 [['1', 'am', 'Your', 'Brother'], ['Thank', 'you'], [...], [...], [...]]
sentences = []
for para in para_list:
    sentences.append(para.split())

In [9]:
from gensim.models import Word2Vec

In [30]:
num_features = 1000  # 词向量维度
min_word_count = 1  # 最小词频数目
num_workers = 40  # 并行线程
context = 10  # Context window size

# Initialize and train the model (this will take some time)

model = Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context)

# It can be helpful to create a meaningful model name and
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "1000features"
model.save(model_name)

In [59]:
model.wv.vocab

{'I': <gensim.models.keyedvectors.Vocab at 0x7ff8a903a0b8>,
 'have': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779358>,
 'deep': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779278>,
 'interest': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779208>,
 'in': <gensim.models.keyedvectors.Vocab at 0x7ff6fb7792e8>,
 'AI': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779198>,
 'and': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779128>,
 'want': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779048>,
 'to': <gensim.models.keyedvectors.Vocab at 0x7ff6fb7793c8>,
 'start': <gensim.models.keyedvectors.Vocab at 0x7ff6fb7798d0>,
 'learning': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779b00>,
 'how': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779b70>,
 'implement': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779be0>,
 'current': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779c50>,
 'method': <gensim.models.keyedvectors.Vocab at 0x7ff6fb779cc0>,
 '.': <gensim.models.keyedvectors.Vocab at 0x7f

In [61]:
model.wv.most_similar("IllegalArgumentException")

[('IllegalStateException', 0.8256390690803528),
 ('UnsupportedOperationException', 0.8167725801467896),
 ('AssertionError', 0.8115333914756775),
 ('IOException', 0.8112595081329346),
 ('IndexOutOfBoundsException', 0.8049449324607849),
 ('ArithmeticException', 0.7900842428207397),
 ('exception', 0.776117205619812),
 ('ArrayIndexOutOfBoundsException', 0.7469482421875),
 ('SQLException', 0.7437758445739746),
 ('NPE', 0.7363903522491455)]

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer

# corpus = ["This is very strange",
#           "This is very nice",
#          ]
vectorizer = TfidfVectorizer(
                        use_idf=True, # utiliza o idf como peso, fazendo tf*idf
                        norm=None, # normaliza os vetores
                        smooth_idf=False, #soma 1 ao N e ao ni => idf = ln(N+1 / ni+1)
                        sublinear_tf=False, #tf = 1+ln(tf)
                        binary=False,
                        min_df=1, max_df=1.0, max_features=None,
                        ngram_range=(1,1), preprocessor=None,
                        stop_words=None, 
                        tokenizer=None,
                        vocabulary=None
             )
X = vectorizer.fit_transform(para_list)
idf = vectorizer.idf_
idf_dict = dict(zip(vectorizer.get_feature_names(), idf))
len(idf_dict)

193952

构建一个小的子集进行原型开发

In [78]:
class Post():

    def __init__(self, title_word_list, answer_list):
        self.title_word_list = title_word_list
        self.answer_list = answer_list
        self.score = 0

    def formula(self, model, q_word_list, Q_word_list, idf_metric_dict):
        rel_idf_list = []
        idf_list = []
        for q_word in q_word_list:
            rel_list = []
            for Q_word in Q_word_list:
                try:
                    val = model.wv.similarity(q_word, Q_word)
                except:
                    val = 0
                rel_list.append(val)

            idf = 0
            try:
                idf = idf_metric_dict[q_word]
            except:
                idf = 0

            rel_idf_list.append(max(rel_list) * idf)
            idf_list.append(idf)
        
        sum_idf =  sum(idf_list)
        if not sum_idf:
            rel_q2Q = 0
        else:
            rel_q2Q = sum(rel_idf_list) / sum_idf

        return rel_q2Q

    def calculate_score(self, model, query_list, idf_metric_dict):
        rel_q2Q = self.formula(model, query_list, self.title_word_list, idf_metric_dict)
        print("----")
        rel_Q2q = self.formula(model, self.title_word_list, query_list, idf_metric_dict)
        self.score = (rel_q2Q + rel_Q2q) / 2
    

In [79]:
post = Post(['I', 'like', 'java'], [])
query_list = ['JAVA']
post.calculate_score(model, query_list, idf_dict)

rel_list: [0.13791901, -0.029415168, 0.6949483]
idf:[0]
rel_idf_list:[0.0]
----
rel_list: [0.13791901]
rel_list: [-0.029415168]
rel_list: [0.6949483]
idf:[0, 3.321801807133636, 2.922064257383658]
rel_idf_list:[0.0, -0.09771135778151795, 2.0306836338034135]


KeyError: 'JAVA'